In [1]:
import pandas as pd
import numpy as np
import requests
import reedsecrets
from requests.auth import HTTPBasicAuth

In [2]:
areas = {'D2N2':['derbyshire','lincolnshire','nottinghamshire','leicestershire'],'NTCA':['newcastle','northumberland'],'CPCA':['Peterborough','Fenland','East Cambridgeshire','South Cambridgeshire','Cambridge' ],'South Yorkshire':['Doncaster','Sheffield','Barnsley','Rotherham']}
courses = {'Front end':['Frontend Developer','UI/UX Developer','web accessibility developer'],'Data Analyst':['Junior Data Analyst','Data Analyst','PowerBI','SQL'],\
          'Business Analysis':['Junior Business Analyst','Business Analyst','strategic analyst','product analyst','business intelligence analyst','project manager','project analyst','program manager','program analyst'],\
                              'Full stack':['Junior Full Stack Developer','Full stack developer']}


In [3]:
token = reedsecrets.token

In [4]:
what = 'UI/UX'
where = 'Newcastle'
baseurl = 'https://www.reed.co.uk/api/1.0/search'
params = { 'minimumSalary':0,'locationName': where, 'keywords': what, 'resultsToTake': 100, 'distanceFromLocation': 10}
resp = requests.get(baseurl, params=params, auth=HTTPBasicAuth(token, ''))
data = resp.json()

In [5]:
resp.json()

{'results': [{'jobId': 50305158,
   'employerId': 432282,
   'employerName': 'Oscar Technology',
   'employerProfileId': None,
   'employerProfileName': None,
   'jobTitle': 'UX/UI Designer - Outside IR35',
   'locationName': 'Newcastle Upon Tyne',
   'minimumSalary': 400.0,
   'maximumSalary': 500.0,
   'currency': 'GBP',
   'expirationDate': '09/06/2023',
   'date': '27/04/2023',
   'jobDescription': 'Outside IR35 Newcastle 3 month contract 5 days on site 300 - 400 p/d A Digital agency, is searching for an experienced UX/UI Designer to work on a variety of client facing projects. Experience needed: UX/ UI Wireframing/ Prototyping HTML/CSS (advantageous) Figma (or alternative) Oscar Associates (UK) Limited is acting as an Employment Business in relation to this vacancy. To understand more about what we do with your data please review our priva... ',
   'applications': 55,
   'jobUrl': 'https://www.reed.co.uk/jobs/ux-ui-designer-outside-ir35/50305158'},
  {'jobId': 50479028,
   'employ

In [8]:
import requests
import pandas as pd
from requests.auth import HTTPBasicAuth

def reed_scraper(what, where, token):
    baseurl = 'https://www.reed.co.uk/api/1.0/search'
    params = {'minimumSalary':0,'maximumSalary': 40000, 'locationName': where, 'keywords': what, 'resultsToTake': 100, 'resultsToSkip': 0, 'distanceFromLocation': 10}
    
    all_results = []
    results_to_take = 100
    
    while True:
        resp = requests.get(baseurl, params=params, auth=HTTPBasicAuth(token, ''))
        data = resp.json()
        
        totalResults = data['totalResults']
        results = data['results']
        
        all_results.extend(results)
        
        if len(all_results) >= totalResults:
            break
            
        params['resultsToSkip'] += len(results)
        
    df = pd.DataFrame(all_results)
    df.drop(['jobId', 'employerId', 'employerProfileId', 'employerProfileName', 'minimumSalary', 'maximumSalary', 'currency', 'expirationDate', 'applications'], axis=1, inplace=True)
    df.columns = ['company', 'title', 'location', 'date', 'description', 'link']
    df['search_term'] = what
    df['area_term'] = where
    
    return df


In [9]:
NTCA['Front end']

,company,title,location,date,description,link,search_term,area_term
0,Oscar Technology,Frontend Developer - Remote - Healthcare - Up ...,Newcastle Upon Tyne,16/05/2023,Vacancy: Frontend Developer Skills: TypeScript...,https://www.reed.co.uk/jobs/frontend-developer...,Frontend Developer,newcastle


In [69]:
reed_scraper('UI/UX developer','Newcastle',token)

,company,title,location,date,description,link,search_term,area_term
0,Noir,React Developer,Newcastle Upon Tyne,15/05/2023,React Developer React Developer - Social Messa...,https://www.reed.co.uk/jobs/react-developer/50...,UI/UX developer,Newcastle
1,Noir,Front End Developer,Gateshead,15/05/2023,Front End Developer Front End Developer - UK T...,https://www.reed.co.uk/jobs/front-end-develope...,UI/UX developer,Newcastle
2,Nigel Wright,React Developer,NE83DF,10/05/2023,Overview Our client&nbsp;is the leading provid...,https://www.reed.co.uk/jobs/react-developer/50...,UI/UX developer,Newcastle
3,Oscar Technology,Frontend Developer - Remote - Healthcare - Up ...,Newcastle Upon Tyne,16/05/2023,Vacancy: Frontend Developer Skills: TypeScript...,https://www.reed.co.uk/jobs/frontend-developer...,UI/UX developer,Newcastle
4,Central Employment Agency (North East) Limited,Content Lead Designer,Newcastle Upon Tyne,24/04/2023,Central Employment are working partnership wit...,https://www.reed.co.uk/jobs/content-lead-desig...,UI/UX developer,Newcastle
5,MFK Recruitment,.NET Developer,NE461AF,28/03/2023,An exciting opportunity has arisen for a .NET ...,https://www.reed.co.uk/jobs/net-developer/5010...,UI/UX developer,Newcastle
6,Careers Plus,Digital Product Owner,DH32SB,22/05/2023,Job Title: Digital Product Owner/Manager Locat...,https://www.reed.co.uk/jobs/digital-product-ow...,UI/UX developer,Newcastle


In [7]:

NTCA = dict()
for x in areas['NTCA']:
    
    for i in courses.keys():
        for a in courses[i]:
            if i not in NTCA:
                df = reed_scraper(a,x,token)
                NTCA[i] = df
                
            else:
                try:
                    
                    NTCA[i] = NTCA[i].append(reed_scraper(a,x,token))
                except:
                    print('Erroooorrrr woohooo',i)
                    pass
                
D2N2 = dict()
for x in areas['D2N2']:
    
    for i in courses.keys():
        for a in courses[i]:
            if i not in D2N2:
                df = reed_scraper(a,x,token)
                D2N2[i] = df
                
            else:
                try:
                    
                    D2N2[i] = D2N2[i].append(reed_scraper(a,x,token))
                except:
                    print('Erroooorrrr woohooo',i)
                    pass
            
    
    

Erroooorrrr woohooo Front end
Erroooorrrr woohooo Front end
Erroooorrrr woohooo Data Analyst
Erroooorrrr woohooo Data Analyst
Erroooorrrr woohooo Data Analyst
Erroooorrrr woohooo Business Analysis
Erroooorrrr woohooo Business Analysis
Erroooorrrr woohooo Business Analysis
Erroooorrrr woohooo Business Analysis
Erroooorrrr woohooo Business Analysis
Erroooorrrr woohooo Business Analysis
Erroooorrrr woohooo Business Analysis
Erroooorrrr woohooo Business Analysis


KeyError: "['jobId', 'employerId', 'employerProfileId', 'employerProfileName', 'minimumSalary', 'maximumSalary', 'currency', 'expirationDate', 'applications'] not found in axis"

In [53]:
NTCA['Data Analyst']['description']

0    Title: Data Analyst Apprentice Salary: Salarie...
1    Apprentices are in demand.&nbsp; Whether you'r...
Name: description, dtype: object